In [1]:
import os; os.chdir('..')

In [2]:
import os
#import shutil
import pandas as pd


In [3]:
import pandas as pd
dataset= pd.read_csv("data/keyword_intent.csv")
dataset.sample(5)

,keyword,intent
408,Discounted gaming laptops,Commercial
10,cat shelves,Transactional
87,DJ khaled,Informational
182,Trulia,Navigational
305,Trulia Business login,Navigational


In [4]:
reverse_intent= {
    'Informational':0,
    'Navigational':1,
    'Transactional':2,
    'Commercial':3,
    'Local':4
}

In [5]:
dataset.intent= pd.Series([reverse_intent[i] for i in dataset.intent])
dataset

,keyword,intent
0,citalopram vs prozac,3
1,who is the oldest football player,0
2,t mobile town east,1
3,starbucks,1
4,tech crunch,1
...,...,...
1066,How to make a paper flower?,0
1067,Why do some animals camouflage?,0
1068,What is the history of ancient civilizations?,0
1069,How to make a simple machine?,0


In [6]:
model_name= "bert-base-cased"


In [7]:
from transformers import AutoTokenizer
tokenizer= AutoTokenizer.from_pretrained(model_name)

/home/ubuntu/intent_generation/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from datasets import load_dataset, Dataset
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['keyword', 'intent'],
    num_rows: 1071
})

In [9]:
dataset_dict= dataset.train_test_split(0.3, train_size=0.7, shuffle=True)

In [10]:
def tokenize_function(examples):
    return tokenizer(examples["keyword"], padding="max_length", truncation=True)

In [11]:
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

Map: 100%|██████████| 322/322 [00:00<00:00, 4600.10 examples/s]


In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['keyword', 'intent', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 749
    })
    test: Dataset({
        features: ['keyword', 'intent', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 322
    })
})

In [12]:
df_train= tokenized_datasets['train']
df_train

Dataset({
    features: ['keyword', 'intent', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 749
})

In [23]:
df_train[100]

{'keyword': 'Theater performances in Broadway NYC',
 'intent': 4,
 'input_ids': [101,
  5978,
  3853,
  1107,
  5259,
  17520,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [13]:
df_test= tokenized_datasets['test']
df_test

Dataset({
    features: ['keyword', 'intent', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 322
})

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

/home/ubuntu/intent_generation/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a 

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

2023-10-04 08:05:02.208399: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 08:05:02.963465: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_train,
    eval_dataset=df_test,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

/home/ubuntu/intent_generation/venv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.